# 딕셔너리와 집합

In [18]:
my_dict = {}
import collections
isinstance(my_dict, collections.abc.Mapping)

True

In [19]:
tt = (1, 2, (30, 40))
print(hash(tt))
# 튜플은 들어 있는 항목들이 모두 해시 가능해야 해시 가능하다.
tl = (1, 2, [30, 40])
hash(tl)

-3907003130834322577


TypeError: unhashable type: 'list'

In [ ]:
tf = (1, 2, frozenset([30, 40]))
hash(tf)

5149391500123939311

## 지능형 딕셔너리

In [ ]:
DIAL_CODES = [
    (86, 'China'),
    (91, 'India'),
    (92, 'Indonesia'),
    (93, 'Brazil'),
    (1, 'Canada'),
    (33, 'United States'),
    (880, 'United States Pacific'),
    (881, 'Russia'),
    (882, 'South Africa'),
    (883, 'South Asia'),
    (884, 'South Pacific'),
    (885, 'South Europe'),
]

country_codes = {country: code for code, country in DIAL_CODES}
country_codes

{'China': 86,
 'India': 91,
 'Indonesia': 92,
 'Brazil': 93,
 'Canada': 1,
 'United States': 33,
 'United States Pacific': 880,
 'Russia': 881,
 'South Africa': 882,
 'South Asia': 883,
 'South Pacific': 884,
 'South Europe': 885}

In [ ]:
""" 단어가 나타나는 위치를 가리키는 인덱스를 만든다."""
import sys
import re
import collections

WORD_RE = re.compile(r'\w+')

# index = {}
index = collections.defaultdict(list)

with open(sys.argv[1], encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start()+1
            location = (line_no, column_no)
            # defaultdict 객체를 활용해서 더욱 간단하게 연산할 수 있다.
            index[word].append(location)
            # index.setdefault(word, []).append(location)
            # 아래 코드는 변경된 occurrences를 딕셔너리에 넣음으로써 index를 한번 더 검색한다.
            """
            occurrences = index.get(word, [])
            occurrences.append(location)
            index[word] = occurrences

            """

for word in sorted(index, key=str.upper):
    print(word, index[word])


In [ ]:
my_dict.setdefault(key, []).append(new_value)
# 위와 아래는 동일한 코드지만 setdefault 함수를 사용하면 단 한번만 검색한다.
if key not in my_dict:
    my_dict[key] = []
my_dict[key].append(new_value)

### \_\_missing__()메서드

In [ ]:
class StrKeyDict0(dict):
    def __missing__(self, key):
        if isinstance(key, str): # 없으면 무한 루프에 빠짐
            raise KeyError(key)
        return self[str(key)]
    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError:
            return default
    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys() #?

In [ ]:
d = StrKeyDict0([('2', 'two'), ('4', 'four')])
d['2']

'two'

In [ ]:
d[4]

'four'

In [ ]:
d[1]

KeyError: '1'

In [ ]:
d.get('2')

'two'

In [ ]:
d.get(4)

'four'

In [ ]:
d.get(1, 'N/A')

'N/A'

In [ ]:
2 in d, 1 in d

(True, False)

### UserDict 상속하기

In [20]:
import collections

class StrKeyDict(collections.UserDict):
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    def __contains__(self, key):
        return str(key) in self.data
    def __setitem__(self, key, item):
        self.data[str(key)] = item

### 불변 매핑

In [21]:
from types import MappingProxyType
d = {1: 'A'}
d_proxy = MappingProxyType(d)
d_proxy, d_proxy[1]

(mappingproxy({1: 'A'}), 'A')

In [22]:
d_proxy[2] = 'x'

TypeError: 'mappingproxy' object does not support item assignment

In [23]:
d[2] = 'B'
d_proxy, d_proxy[2]

(mappingproxy({1: 'A', 2: 'B'}), 'B')

### 집합 이론
집합은 고유한 객체의 모음으로서, 기본적으로 **중복 항목을 제거**한다.   
집합 요소는 반드시 해시할 수 있어야 한다.

In [24]:
l = ['spam', 'spam', 'eggs', 'spam']
set(l)
list(set(l))

['spam', 'eggs']

In [27]:
# 3-11
haystack = {1,2,3,4,5}
needles = {3,4,5,6,7}
found = 0
for n in needles:
    if n in haystack:
        found += 1
found

3

In [25]:
# 3-10
# 위 셀보다 실행 속도가 약간 더 빠르다.
found = len(needles & haystack)
found

3

In [ ]:
# 이렇게 해도 3-11 보다는 빠르다.
found = len(set(needles) * set(haystack))

# 또 다른 방법:
found = len(set(needles).intersection(haystack))

{1}, {1,2} 와 같이 리터럴로 표기하는 것이 set([1,2,3]) 처럼 생성자를 호출하는 것보다 빠르고 가독성이 좋다.   
단 공집합의 경우 set() 구문을 사용해야한다.

In [29]:
from dis import dis

dis('{1}')
dis('set([1])')

  1           0 LOAD_CONST               0 (1)
              2 BUILD_SET                1
              4 RETURN_VALUE
  1           0 LOAD_NAME                0 (set)
              2 LOAD_CONST               0 (1)
              4 BUILD_LIST               1
              6 CALL_FUNCTION            1
              8 RETURN_VALUE


### 지능형 집합

In [30]:
from unicodedata import name
{chr(i) for i in range(32, 256) if 'SIGN' in name(chr(i), '')}

{'#',
 '$',
 '%',
 '+',
 '<',
 '=',
 '>',
 '¢',
 '£',
 '¤',
 '¥',
 '§',
 '©',
 '¬',
 '®',
 '°',
 '±',
 'µ',
 '¶',
 '×',
 '÷'}

In [35]:
d1 = dict(DIAL_CODES)
print(d1, d1.keys())
d2 = dict(sorted(DIAL_CODES))
print(d2, d2.keys())
d3 = dict(sorted(DIAL_CODES, key=lambda x:x[1]))
print(d3, d3.keys)

assert d1 == d2 and d2 == d3

{86: 'China', 91: 'India', 92: 'Indonesia', 93: 'Brazil', 1: 'Canada', 33: 'United States', 880: 'United States Pacific', 881: 'Russia', 882: 'South Africa', 883: 'South Asia', 884: 'South Pacific', 885: 'South Europe'} dict_keys([86, 91, 92, 93, 1, 33, 880, 881, 882, 883, 884, 885])
{1: 'Canada', 33: 'United States', 86: 'China', 91: 'India', 92: 'Indonesia', 93: 'Brazil', 880: 'United States Pacific', 881: 'Russia', 882: 'South Africa', 883: 'South Asia', 884: 'South Pacific', 885: 'South Europe'} dict_keys([1, 33, 86, 91, 92, 93, 880, 881, 882, 883, 884, 885])
{93: 'Brazil', 1: 'Canada', 86: 'China', 91: 'India', 92: 'Indonesia', 881: 'Russia', 882: 'South Africa', 883: 'South Asia', 885: 'South Europe', 884: 'South Pacific', 33: 'United States', 880: 'United States Pacific'} <built-in method keys of dict object at 0x0000022D390FC200>
